In [22]:
from pymongo import MongoClient
import pandas as pd

In [100]:
def get_dataframe(symbol: str, last_n: int):
    try:
        # Connect to MongoDB
        client = MongoClient("mongodb://127.0.0.1:27017/")
        db = client["stockdata"]
        collection_stocks = db["stocks"]
        collection_interest = db["interest"]

        # Aggregation pipeline to get the last n entries for a specific stock
        pipeline = [
            {"$sort": {"_id": -1}},
            {"$limit": last_n},
            {"$unwind": "$bist"},
            {"$match": {"bist.Symbol": symbol}},
            {"$project": {
                "_id": 1,
                "symbol": "$bist.Symbol",
                "volume": "$bist.Volume",
                "open": "$bist.Open",
                "low": "$bist.Low",
                "high": "$bist.High",
                "close": "$bist.Close"
            }}
        ]

        # Execute the aggregation pipeline
        result_stocks = list(collection_stocks.aggregate(pipeline))

        # Create a Pandas DataFrame for stocks
        df_stocks = pd.DataFrame(result_stocks)

        # Sort the DataFrame based on _id
        df_stocks.sort_values("_id", ascending=False, inplace=True)

        # Fetch interest rates from the "interest" collection
        result_interest = list(collection_interest.find({}, {"_id": 1, "Rate": 1}))
        df_interest = pd.DataFrame(result_interest)

        # Merge the dataframes based on the "_id" column
        df_combined = pd.merge(df_stocks, df_interest, left_on="_id", right_on="_id", how="left")

        # Check for duplicate _id values in the merged DataFrame
        duplicate_ids = df_combined[df_combined.duplicated("_id", keep=False)]['_id']
        if not duplicate_ids.empty:
            print(f"Warning: Duplicate _id values found in the merged DataFrame: {duplicate_ids}")

        # Reset index without creating a new index column
        df_combined.reset_index(drop=True, inplace=True)

        return df_combined

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Close the MongoDB connection
        client.close()


In [101]:
df = get_dataframe('TURSG.IS',50)

2     2024-02-23
35    2024-01-09
36    2024-01-09
45    2023-12-26
46    2023-12-26
Name: _id, dtype: object


In [102]:
df.head(30)

,_id,symbol,volume,open,low,high,close,Rate
0,2024-02-26,TURSG.IS,542672.0,50.650002,49.840000,50.799999,50.299999,45.0
1,2024-02-23,TURSG.IS,5101495.0,51.549999,49.000000,51.250000,50.349998,42.5
2,2024-02-23,TURSG.IS,5101495.0,50.650002,49.000000,51.250000,50.349998,42.5
3,2024-02-22,TURSG.IS,4381130.0,51.549999,50.250000,52.200001,50.549999,42.5
4,2024-02-21,TURSG.IS,6001939.0,54.000000,50.500000,54.000000,51.000000,42.5
5,2024-02-20,TURSG.IS,9171466.0,52.799999,51.200001,54.650002,52.599998,42.5
6,2024-02-19,TURSG.IS,3937069.0,56.000000,54.849998,58.000000,55.000000,42.5
7,2024-02-16,TURSG.IS,3884907.0,53.599998,53.549999,55.400002,55.400002,42.5
8,2024-02-15,TURSG.IS,2620666.0,54.400002,53.299999,55.000000,53.299999,42.5
9,2024-02-14,TURSG.IS,3759794.0,52.650002,51.599998,54.299999,53.400002,42.5
